# Passage Pathing

The analysis that follows pertains to the twelfth day of the [Python Problem-Solving Bootcamp](https://mathspp.com/pythonbootcamp).

In the analysis that follows you may be confronted with code that you do not understand, especially as you reach the end of the explanation of each part.

If you find functions that you didn't know before, remember to [check the docs](https://docs.python.org/3/) for those functions and play around with them in the REPL.
This is written to be increasing in difficulty (within each part of the problem), so it is understandable if it gets harder as you keep reading.
That's perfectly fine, you don't have to understand everything _right now_, especially because I can't know for sure what _your level_ is.

## Part 1 problem statement

(From [Advent of Code 2021, day 12](https://adventofcode.com/2021/day/12))


With your submarine's subterranean subsystems subsisting suboptimally, the only way you're getting out of this cave anytime soon is by finding a path yourself. Not just _a_ path - the only way to know if you've found the _best_ path is to find _all_ of them.

Fortunately, the sensors are still mostly working, and so you build a rough map of the remaining caves (your puzzle input). For example:

```
start-A
start-b
A-c
A-b
b-d
A-end
b-end
```

This is a list of how all of the caves are connected. You start in the cave named `start`, and your destination is the cave named `end`. An entry like `b-d` means that cave `b` is connected to cave `d` - that is, you can move between them.

So, the above cave system looks roughly like this:

```
    start
    /   \
c--A-----b--d
    \   /
     end
```

Your goal is to find the number of distinct _paths_ that start at `start`, end at `end`, and don't visit small caves more than once. There are two types of caves: _big_ caves (written in uppercase, like `A`) and _small_ caves (written in lowercase, like `b`). It would be a waste of time to visit any small cave more than once, but big caves are large enough that it might be worth visiting them multiple times. So, all paths you find should _visit small caves at most once_, and can _visit big caves any number of times_.

Given these rules, there are `10` paths through this example cave system:

```
start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,end
start,A,c,A,b,A,end
start,A,c,A,b,end
start,A,c,A,end
start,A,end
start,b,A,c,A,end
start,b,A,end
start,b,end
```

(Each line in the above list corresponds to a single path; the caves visited by that path are listed in the order they are visited and separated by commas.)

Note that in this cave system, cave `d` is never visited by any path: to do so, cave `b` would need to be visited twice (once on the way to cave `d` and a second time when returning from cave `d`), and since cave `b` is small, this is not allowed.

Here is a slightly larger example:

```
dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc
```

The `19` paths through it are as follows:

```
start,HN,dc,HN,end
start,HN,dc,HN,kj,HN,end
start,HN,dc,end
start,HN,dc,kj,HN,end
start,HN,end
start,HN,kj,HN,dc,HN,end
start,HN,kj,HN,dc,end
start,HN,kj,HN,end
start,HN,kj,dc,HN,end
start,HN,kj,dc,end
start,dc,HN,end
start,dc,HN,kj,HN,end
start,dc,end
start,dc,kj,HN,end
start,kj,HN,dc,HN,end
start,kj,HN,dc,end
start,kj,HN,end
start,kj,dc,HN,end
start,kj,dc,end
```

Finally, this even larger example has `226` paths through it:

```
fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW
```

_How many paths through this cave system are there that visit small caves at most once?_

_Using the input file `input.txt`, the result should be `3410`._

In [1]:
# IMPORTANT: Set this to the correct path for you!
INPUT_FILE = "data/input.txt"

## Representing the graph as a list of edges

When you have a series of points (or positions, or places) that are connected amongst each other, you have a graph.
A graph is a (mathematical) object composed of a set of vertices and a set of edges, where the edges represent connections vertices.
Graphs crop up in all sorts of domains because your set of vertices can be anything, and your set of edges can be any sort of relationship between the vertices.

Sometimes, it is obvious that you are dealing with a graph.
Other times, it's less obvious.
This time, it's quite clear that we have a graph at hands.

Graphs can be represented in many different ways, and you have to pick the most convenient way for you, whenever you work with a graph.

The first way we are going to go over is the [list of edges](https://en.wikipedia.org/wiki/Edge_list), which corresponds to the way the input is given in this problem:

In [2]:
with open(INPUT_FILE, "r") as f:
    graph_edges = [line.strip().split("-") for line in f.readlines()]
graph_edges

[['yb', 'start'],
 ['de', 'vd'],
 ['rj', 'yb'],
 ['rj', 'VP'],
 ['OC', 'de'],
 ['MU', 'de'],
 ['end', 'DN'],
 ['vd', 'end'],
 ['WK', 'vd'],
 ['rj', 'de'],
 ['DN', 'vd'],
 ['start', 'VP'],
 ['DN', 'yb'],
 ['vd', 'MU'],
 ['DN', 'rj'],
 ['de', 'VP'],
 ['yb', 'OC'],
 ['start', 'rj'],
 ['oa', 'MU'],
 ['yb', 'de'],
 ['oa', 'VP'],
 ['jv', 'MU'],
 ['yb', 'MU'],
 ['end', 'OC']]

When representing a graph as a list of edges, it is generally helpful to split the edge into the two vertices.
Sometimes, the edges are oriented (which means that information only flows in one direction) and sometimes the edge has no orientation.

For example, in a family tree, the direction of an arrow can be used to represent parenthood.
Thus, if `A` is `B`'s parent, certainly `B` can't be `A`'s parent.
On the other hand, if edges represent connections between cities, if a city `A` is connected to a city `B`, then the city `B` is also connected to the city `A`.

In our case, we don't have oriented edges (because if a cave `A` is connected to a cave `B`, then the cave `B` is also connected to the cave `A`).

The problem statement concerns itself with navigating through a cave system, so, given this graph representation, how could we figure out the caves that can be visited from a specific cave?

Let's write some code:

In [3]:
def neighbours(graph, cave):
    neighbs = []
    for edge in graph:
        if cave in edge:
            neighbs.append(edge[0] if edge[1] == cave else edge[1])
    return neighbs

In [4]:
neighbours(graph_edges, "start")

['yb', 'VP', 'rj']

This seems like a promising start.

Now, there are three things we can do to our graph representation:

 - do nothing;
 - reverse each edge and add it to the list; or
 - change representation.

The first option has the advantage that we have no more work to do now, although we might pay that later if it turns out that this representation has any defficiencies.

The second option has the advantage that it becomes easier to figure out where one can go from a single cave:

In [5]:
with open(INPUT_FILE, "r") as f:
    graph_edges = [line.strip().split("-") for line in f.readlines()]
graph_edges += [edge[::-1] for edge in graph_edges]

def neighbours(graph, cave):
    return [dest for source, dest in graph if source == cave]

neighbours(graph_edges, "start")

['VP', 'rj', 'yb']

The third option has the advantage that we get to play around with more graph representations.
Let's do that.

## Representing the graph as an adjacency "matrix"

Another common way of representing a graph is through an [adjancecy matrix](https://en.wikipedia.org/wiki/Adjacency_matrix).
An adjacency matrix is a matrix where each row and column represents a vertex, or a node.
If there is an edge between two vertices, the corresponding entry in the matrix is set to one.
If there is no edge between two vertices, the corresponding entry in the matrix is set to zero.

We have worked a bit with 2D arrays already, but let's go back to an idea from the previous analysis and use a dictionary to represent our matrix.

After reading in the edge list, we can figure out all the vertices in our graph and initialise the adjacency matrix for the graph.
Then, finding the neighbours of a single vertex is a matter of looking for positions in the matrix that are set to true:

In [6]:
from itertools import product

with open(INPUT_FILE, "r") as f:
    graph_edges = [line.strip().split("-") for line in f.readlines()]

# Read a set of nodes to remove duplicates and convert to list to preserve order.
NODES = list({node for edge in graph_edges for node in edge})

graph = dict.fromkeys(product(NODES, NODES), False)
for v1, v2 in graph_edges:
    graph[(v1, v2)] = True
    graph[(v2, v1)] = True

def neighbours(graph, cave):
    return [dest for dest in NODES if graph[(cave, dest)]]

neighbours(graph, "start")

['yb', 'rj', 'VP']

This is an interesting of representing the graph, but adjacency matrices are typically more useful when we can actually use a _matrix_ to do some computations with it.
Thus, this fake matrix representation with a dictionary isn't going to work very well, but it may lead us into a better representation.

## Representing the graph as an adjacency list

Instead of keeping a dictionary which contains edges as keys and Boolean values as, well, values, let's do something more useful.

We don't want to have to iterate over the whole list `NODES` just to find out what vertices are adjacent to a given vertex.
Instead of coupling the vertices like that in the keys to the dictionary `graph`, let's have a key _per vertex_, and then each vertex is associated with a list of all the vertices that are connected to it.

This is what's commonly called an [adjacency list](https://en.wikipedia.org/wiki/Adjacency_list) and makes it _very_ convenient to traverse the graph.
Instead of needing to call the function `neighbours`, we can just use a key to access the graph:

In [7]:
with open(INPUT_FILE, "r") as f:
    graph_edges = [line.strip().split("-") for line in f.readlines()]

graph = {}
for v1, v2 in graph_edges:
    if v1 not in graph:
        graph[v1] = []
    if v2 not in graph:
        graph[v2] = []
    graph[v1].append(v2)
    graph[v2].append(v1)

graph["start"]

['yb', 'VP', 'rj']

In [8]:
graph

{'yb': ['start', 'rj', 'DN', 'OC', 'de', 'MU'],
 'start': ['yb', 'VP', 'rj'],
 'de': ['vd', 'OC', 'MU', 'rj', 'VP', 'yb'],
 'vd': ['de', 'end', 'WK', 'DN', 'MU'],
 'rj': ['yb', 'VP', 'de', 'DN', 'start'],
 'VP': ['rj', 'start', 'de', 'oa'],
 'OC': ['de', 'yb', 'end'],
 'MU': ['de', 'vd', 'oa', 'jv', 'yb'],
 'end': ['DN', 'vd', 'OC'],
 'DN': ['end', 'vd', 'yb', 'rj'],
 'WK': ['vd'],
 'oa': ['MU', 'VP'],
 'jv': ['MU']}

There you go, a graph represented as an adjacency list!

Now, in order to implement this in a proper way, we should actually make use of the class `defaultdict` from the module `collections`, which we have seen a couple of times already.
By using the built-in `list` as the object constructor, we can make it so that the `defaultdict` returns an empty list for a key that has never been used:

In [9]:
from collections import defaultdict

with open(INPUT_FILE, "r") as f:
    graph_edges = [line.strip().split("-") for line in f.readlines()]

graph = defaultdict(list)
for v1, v2 in graph_edges:
    graph[v1].append(v2)
    graph[v2].append(v1)

graph["start"]

['yb', 'VP', 'rj']

In [10]:
graph

defaultdict(list,
            {'yb': ['start', 'rj', 'DN', 'OC', 'de', 'MU'],
             'start': ['yb', 'VP', 'rj'],
             'de': ['vd', 'OC', 'MU', 'rj', 'VP', 'yb'],
             'vd': ['de', 'end', 'WK', 'DN', 'MU'],
             'rj': ['yb', 'VP', 'de', 'DN', 'start'],
             'VP': ['rj', 'start', 'de', 'oa'],
             'OC': ['de', 'yb', 'end'],
             'MU': ['de', 'vd', 'oa', 'jv', 'yb'],
             'end': ['DN', 'vd', 'OC'],
             'DN': ['end', 'vd', 'yb', 'rj'],
             'WK': ['vd'],
             'oa': ['MU', 'VP'],
             'jv': ['MU']})

Finally, because the processing we are doing is very basic, we can build the graph directly when we read the data:

In [11]:
from collections import defaultdict

graph = defaultdict(list)
with open(INPUT_FILE, "r") as f:
    for line in f:
        v1, v2 = line.strip().split("-")
        graph[v1].append(v2)
        graph[v2].append(v1)

graph["start"]

['yb', 'VP', 'rj']

Now, in order to solve the actual problem, we need to employ a search algorithm in our graph.

When working with graphs, there are many algorithms to compute specific things with those graphs.
For example, you can try finding the shortest distance between two vertices, or you might want to compute a tree that represents your graph, or you might want to check if the graph has cycles, or ...

However, there are two basic algorithms that are useful for when you need to do some generic work in/with a graph, and those two algorithms are breadth-first search and depth-first search.
We will use both to solve the problem.

## Breadth-first search (BFS)

The idea of [breadth-first search](https://en.wikipedia.org/wiki/Breadth-first_search) (BFS) is that, when you reach a new vertex, you first take a look at all the vertices that surround you, and only then deepen your search by following into a new vertex.

This is to contrast with [depth-first search](https://en.wikipedia.org/wiki/Depth-first_search), because when you reach a new vertex, you immediately start exploring deeper into the graph before looking around you and checking what options you have available.

Typically, we use a search algorithm to find something specific.
However, in this case, we just want to explore the whole graph and count how many paths we find.
We can frame this as searching for the vertex labeled `"end"`.

A BFS implementation follows:

In [35]:
from collections import defaultdict, deque

graph = defaultdict(list)
with open(INPUT_FILE, "r") as f:
    for line in f:
        v1, v2 = line.strip().split("-")
        graph[v1].append(v2)
        graph[v2].append(v1)
        
count = 0
queue = deque([("start", )])
while queue:
    path_so_far = queue.pop()
    if path_so_far[-1] == "end":
        count += 1
        continue

    for neighbour in graph[path_so_far[-1]]:
        if not (neighbour.lower() == neighbour and neighbour in path_so_far):
            queue.appendleft(path_so_far + (neighbour, ))

print(count)

3410


The algorithm above creates a queue (by leveraging the class `deque` from the module `collections`) and seeds it with the `"start"` cave.
Then, while there are paths to explore in the cave, we pick up the next path.
After we make sure the path isn't finished yet, we explore all possible caves and add the valid paths for later exploration.
A valid path is a path that doesn't contain a repeated lowercase cavern.

The code we wrote above can be cleaned up a bit.
For one, we are checking if the neighbouring cave `neighbour` is a lowercase cave by checking `neighbour.lower() == neighbour`.
This is redundant, because strings have a method `.islower` that checks just that.

Secondly, we can use starred assignment to destructure the last visited vertex into a variable of its own, making the code a bit cleaner.

Lastly, we can use a list comprehension, and the method `.extendleft`, to insert a series of elements into the queue in one fell swoop:

In [50]:
count = 0
queue = deque([["start"]])
while queue:
    path = *_, last_visited = queue.pop()  # <-
    if last_visited == "end":
        count += 1
        continue

    queue.extendleft([
        path + [neighbour] for neighbour in graph[last_visited]  # <-
        if not (neighbour.islower() and neighbour in path)  # <-
    ])

print(count)

3410


BFS is a staple of graph algorithms, but one thing I don't appreciate is that the queue that holds the paths that have to be processed can grow quite a bit.

Switching to DFS can help with that.

## Depth-first search (DFS)

As mentioned above, [depth-first search](https://en.wikipedia.org/wiki/Depth-first_search) is a graph search algorithm that prioritises exhausting search paths over exploring multiple possibilities.
In other words, DFS _first_ tries to complete a path, and only then explores different routes.

By taking our BFS code from above, we can change it from breadh-first into depth-first by a simple change of data structure.
By using a queue in breadth-first search, and by putting new elements on the queue on one side and popping them from the other, we manage to prioritise exploration.
If we switch to a stack, where we put and pop elements from the same end, we will be prioritising finishing a path before exploring new paths.

Here is a possible DFS implementation with a stack:

In [51]:
from collections import defaultdict, deque

graph = defaultdict(list)
with open(INPUT_FILE, "r") as f:
    for line in f:
        v1, v2 = line.strip().split("-")
        graph[v1].append(v2)
        graph[v2].append(v1)
        
count = 0
queue = deque([["start"]])
while queue:
    path = *_, last_visited = queue.pop()
    if last_visited == "end":
        count += 1
        continue

    queue.extend([  # <- changed from `extendleft` to `extend`.
        path + [neighbour] for neighbour in graph[last_visited]
        if not (neighbour.islower() and neighbour in path)
    ])

print(count)

3410


The code is _identical_ to the BFS from above, except the single method call where the comment is, which was changed from `.extendleft` to `.extend`.
Everything else is 100% the same.

Of course it shouldn't, though, because now we are using a stack and not a queue.
Thus, the name of the variable should change:

In [52]:
count = 0
stack = deque([["start"]])
while stack:
    path = *_, last_visited = stack.pop()
    if last_visited == "end":
        count += 1
        continue

    stack.extend([
        path + [neighbour] for neighbour in graph[last_visited]
        if not (neighbour.islower() and neighbour in path)
    ])

print(count)

3410


## Recursive DFS

Recursion and iteration are essentially equivalent, which means that, whatever you did with one, you should be able to do with the other.
Now, of course this is simpler some times and more difficult in other times.
However, when you have some stack-based iteration, that should be easy to rewrite as a recursive function.

For the sake of completeness, let us experiment with a recursive version of DFS, by means of an auxiliary function:

In [4]:
from collections import defaultdict, deque

graph = defaultdict(list)
with open(INPUT_FILE, "r") as f:
    for line in f:
        v1, v2 = line.strip().split("-")
        graph[v1].append(v2)
        graph[v2].append(v1)
        
def dfs(graph, path_so_far):
    *_, last_visited = path_so_far
    if last_visited == "end":
        return 1

    count = 0
    for neighbour in graph[path_so_far[-1]]:
        if not (neighbour.islower() and neighbour in path_so_far):
            count += dfs(graph, path_so_far + [neighbour])
    return count

dfs(graph, ["start"])

3410

This recursive definition is _very_ similar to the one with the `while` loop, except that we do not have to keep track of an explicit stack of options to explore.
The (recursive) call stack takes care of that for us.

Now, at this point you should know by heart that modification I am going to suggest next.
If you look at the end of the function `dfs`, you will notice, once more, the pattern of a reduction!
In particular, of `sum`.

To make use of `sum` there, we just need to use a list comprehension to apply the function `dfs` recursively to the path extended by each of the neighbours:

In [6]:
from collections import defaultdict, deque

graph = defaultdict(list)
with open(INPUT_FILE, "r") as f:
    for line in f:
        v1, v2 = line.strip().split("-")
        graph[v1].append(v2)
        graph[v2].append(v1)
        
def dfs(graph, path_so_far):
    *_, last_visited = path_so_far
    if last_visited == "end":
        return 1

    return sum(
        dfs(graph, path_so_far + [neighbour]) for neighbour in graph[last_visited]
        if not (neighbour.islower() and neighbour in path_so_far)
    )

dfs(graph, ["start"])

3410

## De Morgan's laws

The [De Morgan's laws](https://en.wikipedia.org/wiki/De_Morgan%27s_laws) are two rules that apply to Boolean expressions.
De Morgan's laws state that:

 1. `not (a and b)` is the same as `(not a) or (not b)`; and
 2. `not (a or b)` is the same as `(not a) and (not b)`.

By inserting `not`s here and there, they can be used to play around with Boolean expressions in many different ways.
What I personally find interesting is that De Morgan's laws make very much intuitive sense.

For example, in the above, we only want to consider a new vertex into the path `if not (neighbour.islower() and neighbour in path_so_far)`.
In English words, if it is not the case that the neighbour is a lowercase cave **and** is already in the path.
Of course, we could try to write this in the affirmative.
Such an appropriate refactour could mean “we care about the next vertex if it is an uppercase cave or if it is not in the path yet”, which allows us to rewrite the code as follows:

In [7]:
def dfs(graph, path_so_far):
    *_, last_visited = path_so_far
    if last_visited == "end":
        return 1

    return sum(
        dfs(graph, path_so_far + [neighbour]) for neighbour in graph[last_visited]
        if neighbour.isupper() or neighbour not in path_so_far
    )

dfs(graph, ["start"])

3410

This refactoring of the `if` statement was purely stylistic and I invete the reader to pick whichever version they prefer.
I prefer the current one because it maps out better to English, especially because of the `not in` operator:
recall that the other alternative starts with `not` and then a set of parenthesis to group everything.

On the other hand, the the line of code

```py
if neighbour.isupper() or neighbour not in path_so_far
```

reads directly as “this neighbour matters if it is an uppercase cave or if it hasn't been visited yet, before”.

We've seen two main graph search algorithms and a couple of different alternative implementations, so let's crack the second part now:

## Part 2 problem statement

(From [Advent of Code 2021, day 12](https://adventofcode.com/2021/day/12))

After reviewing the available paths, you realize you might have time to visit a single small cave _twice_. Specifically, big caves can be visited any number of times, a single small cave can be visited at most twice, and the remaining small caves can be visited at most once. However, the caves named `start` and `end` can only be visited _exactly once each_: once you leave the `start` cave, you may not return to it, and once you reach the `end` cave, the path must end immediately.

Now, the `36` possible paths through the first example above are:

```
start,A,b,A,b,A,c,A,end
start,A,b,A,b,A,end
start,A,b,A,b,end
start,A,b,A,c,A,b,A,end
start,A,b,A,c,A,b,end
start,A,b,A,c,A,c,A,end
start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,d,b,A,c,A,end
start,A,b,d,b,A,end
start,A,b,d,b,end
start,A,b,end
start,A,c,A,b,A,b,A,end
start,A,c,A,b,A,b,end
start,A,c,A,b,A,c,A,end
start,A,c,A,b,A,end
start,A,c,A,b,d,b,A,end
start,A,c,A,b,d,b,end
start,A,c,A,b,end
start,A,c,A,c,A,b,A,end
start,A,c,A,c,A,b,end
start,A,c,A,c,A,end
start,A,c,A,end
start,A,end
start,b,A,b,A,c,A,end
start,b,A,b,A,end
start,b,A,b,end
start,b,A,c,A,b,A,end
start,b,A,c,A,b,end
start,b,A,c,A,c,A,end
start,b,A,c,A,end
start,b,A,end
start,b,d,b,A,c,A,end
start,b,d,b,A,end
start,b,d,b,end
start,b,end
```

The slightly larger example above now has `103` paths through it, and the even larger example now has `3509` paths through it.

Given these new rules, _how many paths through this cave system are there?_

_Using the input file `input.txt`, the result should be `98796`._

## Different filtering

Once more (just like with the previous challenge) it seems that a solid answer to the first part goes a _long_ way to solve the second part.

As it turns out, we just need to be slightly more permissive when checking which neighbours are useful.
Now, we can accept a repeated lowercase cave if that's the first time it happens.

To check if that's happened before, we resort to an auxiliary function and check if there is any lowercase cave that has been visited more than once.
On top of that, we have to be careful not to visit the vertices `"start"` or `"end"` twice.

Thankfully, we will never visit the vertex `"end"` twice because, as soon as we reach that vertex, we return.
In order to avoid the vertex `"start"`, we can filter it out:

In [10]:
from collections import Counter, defaultdict, deque  # <-

graph = defaultdict(list)
with open(INPUT_FILE, "r") as f:
    for line in f:
        v1, v2 = line.strip().split("-")
        graph[v1].append(v2)
        graph[v2].append(v1)

def has_repeated_cave(path):
    caves = Counter([cave for cave in path if cave.islower()])
    return max(caves.values()) > 1
        
def dfs(graph, path):
    *_, last_visited = path
    if last_visited == "end":
        return 1

    return sum(
        dfs(graph, path + [neighb]) for neighb in graph[last_visited]
        if neighb != "start" and (neighb.isupper() or neighb not in path or not has_repeated_cave(path))
    )

dfs(graph, ["start"])

98796

Not only is line 21 _very_ long, it is also wasteful.
Notice that `has_repeated_cave(path)` only depends on the path we have so far, and yet we recompute it for every iteration of `neighb` over the possible next neighbours.
In order to save some time, we can compute it before the list comprehension:

In [11]:
def dfs(graph, path):
    *_, last_visited = path
    if last_visited == "end":
        return 1

    can_repeat = not has_repeated_cave(path)
    return sum(
        dfs(graph, path + [neighb]) for neighb in graph[last_visited]
        if neighb != "start" and (neighb.isupper() or neighb not in path or can_repeat)
    )

dfs(graph, ["start"])

98796

This saves some computational power, but we still have a very long line, and very long lines are frowned upon.
If your code takes up too much horizontal space, it also ends up being harder to read!

The line became much longer because we have to filter out the `"start"` cave and because we need to check if we can repeat a lowercase cave, so let's see if we can do something about it.

## Removing the initial cave as a destination

The issue is that many caves view the `"start"` cave as a possible destination, and we have to keep ignoring it over and over again.

Another way of dealing with this would boil down to removing the cave `"start"` from the adjacency list of all other caves, making it impossible to return to the cave `"start"`.
In order to remove an item from an adjacency list (which is represented by the built-in type `list`), we can use the `.remove` method:

In [12]:
l = [1, 2, 3, 4]
l.remove(3)
l

[1, 2, 4]

We just have to be careful with what happens if we call the method `.remove` and the item isn't in the list:

In [13]:
l.remove(5)

ValueError: list.remove(x): x not in list

Thus, we need to prevent the `ValueError` of being thrown:

In [14]:
from collections import Counter, defaultdict, deque

graph = defaultdict(list)
with open(INPUT_FILE, "r") as f:
    for line in f:
        v1, v2 = line.strip().split("-")
        graph[v1].append(v2)
        graph[v2].append(v1)

for connections in graph.values():
    if "start" in connections:
        connections.remove("start")

def has_repeated_cave(path):
    caves = Counter([cave for cave in path if cave.islower()])
    return max(caves.values()) > 1
        
def dfs(graph, path):
    *_, last_visited = path
    if last_visited == "end":
        return 1

    can_repeat = not has_repeated_cave(path)
    return sum(
        dfs(graph, path + [neighb]) for neighb in graph[last_visited]
        if neighb.isupper() or neighb not in path or can_repeat
    )

dfs(graph, ["start"])

98796

By using an `if` statement, we only remove the cave `"start"` from the adjacency lists that had it in the first place.
However, using an `if` statement to look inside the list `connections` before we try to remove something from it follows the LBYL (Look Before You Leap) coding style, and Python often prefers the EAFP (Easier to Ask Forgiveness than Permission) style.

You can read about both styles [in this article](https://mathspp.com/blog/pydonts/eafp-and-lbyl-coding-styles), but the gist of it is that the LBYL is characterised by `if` statements prior to the main action, whereas the EAFP style encloses the main action with a `try` statement.

Hence, following the EAFP style, we could write something like this:

In [15]:
graph = defaultdict(list)
with open(INPUT_FILE, "r") as f:
    for line in f:
        v1, v2 = line.strip().split("-")
        graph[v1].append(v2)
        graph[v2].append(v1)

for connections in graph.values():
    try:
        connections.remove("start")
    except ValueError:
        pass

This is the EAFP coding style and, in this particular case, saves us from going over the list `connections` twice, but it is kind of a shame that it became so much longer...

Thankfully for us, this `try: ... except: ...` block has a very specific pattern: it has a `try`, and the `except` is just a `pass` statement.
What do `try` blocks like that mean?
They mean we want to try to do something, but we don't care if it fails.
For example, in our case, we want to delete the cave `"start"` from all adjacency lists.
But, if the `.remove` call failed, that's because the cave `"start"` wasn't there to begin with, so I don't care about the error, I just want to ignore it.

When in situations like this, there is a nice tool for you in the module `contextlib`, called `suppress`: it's a context manager that let's you suppress (that is, ignore) the exceptions you specify:

In [16]:
from contextlib import suppress  # <-

graph = defaultdict(list)
with open(INPUT_FILE, "r") as f:
    for line in f:
        v1, v2 = line.strip().split("-")
        graph[v1].append(v2)
        graph[v2].append(v1)

for connections in graph.values():
    with suppress(ValueError):  # Ignore ValueError's that might get raised.
        connections.remove("start")

## Carrying information about repeating caves

The last tweak that we want to make to our function has to do with the information regarding repeated caves.
First, for each tentative neighbour, we checked if the path already had repeated caves.
Then, we realised that we can do it before that loop to save some time.

Now, let's think about this: if at a given point in the path we can't repeat a lowercase cave because we already did, will we be able to do it again later on?
No, we won't!
So, as soon as we repeat a cave, we can carry that information with us, because we don't need to recompute it every single time.

To do this, we will employ a very common technique in recursive programming: we pass some information about the state of the program as an argument!
This shouldn't sound mind-blowing, because that's what we are already doing with the list `path`, for example.

So, when we are about to call the function `dfs` recursively, we need to figure out how to update the Boolean flags that tells us if we can repeat a lowercase cave or not.
When preparing the recursive call, we can say that we have repeated a lowercase cave if we already did it before **or** if we just repeated a lowercase cave:

In [20]:
from collections import defaultdict, deque
from contextlib import suppress

graph = defaultdict(list)
with open(INPUT_FILE, "r") as f:
    for line in f:
        v1, v2 = line.strip().split("-")
        graph[v1].append(v2)
        graph[v2].append(v1)

for connections in graph.values():
    with suppress(ValueError):
        connections.remove("start")
        
def dfs(graph, path, has_repeated):
    *_, last_visited = path
    if last_visited == "end":
        return 1

    return sum(
        dfs(graph, path + [neighb], has_repeated or (neighb.islower() and neighb in path))
        for neighb in graph[last_visited]
        if neighb.isupper() or neighb not in path or not has_repeated
    )

dfs(graph, ["start"], False)

98796

With this Boolean flag, we can even solve both tasks with a single function.
For the first problem, we just have to set the Boolean flag `has_repeated` to `True`, as if we had already repeated a lowercase cave:

In [21]:
print(dfs(graph, ["start"], True))   # Part 1.
print(dfs(graph, ["start"], False))  # Part 2.

3410
98796


## Naming matters

We have been doing this implicitly, but now I'll call it out.
[Naming matters](https://mathspp.com/blog/pydonts/naming-matters).
It matters because our brains can only hold a small number of ideas and thoughts at the same time, and giving proper names to things makes it easier to manage them in our heads.

For functions, in particular, the name of the function should tell you what the function _does_, and not _how_ it does it.
So, in our case, I'm talking about the function `dfs`.
The name isn't that great because we used the acronym for the algorithm we implemented, when in fact the function has a very specific purpose: to count paths in a graph.
Thus, our function should've had a name that matched that purpose, instead of a name that reflects the internal algorithm being used.

In our case, something along the lines of `count_paths` would've worked perfectly:

In [23]:
from collections import defaultdict, deque
from contextlib import suppress

graph = defaultdict(list)
with open(INPUT_FILE, "r") as f:
    for line in f:
        v1, v2 = line.strip().split("-")
        graph[v1].append(v2)
        graph[v2].append(v1)

for connections in graph.values():
    with suppress(ValueError):
        connections.remove("start")
        
def count_paths(graph, path, has_repeated):  # <-
    *_, last_visited = path
    if last_visited == "end":
        return 1

    return sum(
        count_paths(
            graph,
            path + [neighb],
            has_repeated or (neighb.islower() and neighb in path),
        )
        for neighb in graph[last_visited]
        if neighb.isupper() or neighb not in path or not has_repeated
    )

count_paths(graph, ["start"], False)

98796

As the name of the function increased, the recursive call line became too long and we had to split it,
that's why you see each argument on its own line now.

## Conclusion

The two classical graph algorithms are _very_ similar, and yet depth-first search is very amenable to a transation into a recursive version, whereas breadth-first search not so much.
We also took the chance to look at a couple of different ways of representing graphs and realised first-hand that choosing your data structures wisely is half the job when solving a problem.

If you have any questions, suggestions, remarks, recommendations, corrections, or anything else, you can reach out to me [on Twitter](https://twitter.com/mathsppblog) or via email to rodrigo at mathspp dot com.